# Four-Circle example (E4C)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import gi
gi.require_version('Hkl', '5.0')

from hkl.calc import CalcE4CV
from hkl.util import Lattice

import pyRestTable

In [2]:
# initialize the calculation engine
fourc_calc = CalcE4CV(engine='hkl')
fourc_calc.engine.mode = 'bissector'
fourc_calc.wavelength = 1.5420925870646767  # Cu Kalpha

In [3]:
# add a sample
fourc_calc.new_sample('Mn3O4/MgO thin film', 
    lattice=Lattice(
        a=5.72, b=5.72, c=9.5, 
        alpha=90.0, beta=90.0, gamma=90.0))


HklSample(name='Mn3O4/MgO thin film', lattice=LatticeTuple(a=5.72, b=5.72, c=9.5, alpha=90.0, beta=90.0, gamma=90.0), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), U=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), UB=array([[ 1.09845897e+00, -6.72612131e-17, -4.04983304e-17],
       [ 0.00000000e+00,  1.09845897e+00, -4.04983304e-17],
       [ 0.00000000e+00,  0.00000000e+00,  6.61387927e-01]]), reflections=[])

In [4]:
# use two known reflections to define the orientation
r1 = fourc_calc.sample.add_reflection(
    -1.998, -1.994, 4.011,
    position=fourc_calc.Position(
        tth=80.8769, omega=40.6148, chi=0.647, phi=-121.717))
r2 = fourc_calc.sample.add_reflection(
    -0.997, -0.997, 2.009,
    position=fourc_calc.Position(
        tth=28.695, omega=14.4651, chi=-48.8860, phi=-88.758))
fourc_calc.sample.compute_UB(r1, r2)

1

In [5]:
# define some constraints
#        "omega": AxisConstraints(-120, 120, 0, True),
#        "chi": AxisConstraints(-120, 120, 0, True),
omega = fourc_calc['omega']
omega.limits = (-120, 120)
omega.value = 0
omega.fit = True

chi = fourc_calc['chi']
chi.limits = (-120, 120)
chi.value = 0
chi.fit = True


In [28]:
# show motor positions at some hkl reflections
reflections = (
    (-2,-2,4), 
    (-1,-1,2), 
    (-2,1,1), 
    (-3,0,5), 
    (0,3,1), 
    (0,3,.5), 
    (0,3,1.5),
)
motors = "omega chi phi tth".split()
tbl = pyRestTable.Table()
tbl.labels = "h k l solution".split() + motors
for ref in reflections:
    for i, sol in enumerate(fourc_calc.forward(ref)):
        row = list(ref) + [i,] + [f"{getattr(sol, m):.4f}" for m in motors]
        tbl.addRow(row)
print(tbl.reST(fmt="simple"))

== == === ======== ======== ======== ========= ========
h  k  l   solution omega    chi      phi       tth     
== == === ======== ======== ======== ========= ========
-2 -2 4   0        -30.0505 -0.6894  58.3246   -60.1009
-2 -2 4   1        30.0505  0.6894   -121.6754 60.1009 
-1 -1 2   0        -14.5001 -0.6894  58.3246   -29.0001
-1 -1 2   1        14.5001  0.6894   -121.6754 29.0001 
-2 1  1   0        18.1891  48.6540  -68.0790  36.3782 
-2 1  1   1        -18.1891 -48.6540 111.9210  -36.3782
-3 0  5   0        -34.9530 -16.9367 87.4333   -69.9061
-3 0  5   1        34.9530  16.9367  -92.5667  69.9061 
0  3  1   0        24.3594  12.2330  -7.3316   48.7188 
0  3  1   1        -24.3594 -12.2330 172.6684  -48.7188
0  3  0.5 0        23.9805  14.9737  -2.2851   47.9610 
0  3  0.5 1        -23.9805 -14.9737 177.7149  -47.9610
0  3  1.5 0        24.9813  9.5105   -12.0878  49.9627 
0  3  1.5 1        -24.9813 -9.5105  167.9122  -49.9627
== == === ======== ======== ======== ========= =